In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
data = spark.read.csv("/FileStore/tables/SMSSpamCollection.csv",inferSchema=True,sep=',')

In [4]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [5]:
data.show(truncate = False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
class|text |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
ham |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat... |
ham |Ok lar... Joking wif u oni... |
spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's |
ham |U dun say so early hor... U c already then say... |
ham |Nah I don't think he goes to usf, he lives around here though |
spam |FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, Â£1.50 to rcv |
ham |Even my brother is not like to speak with me. They treat me like aids patent. |
ham |As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune |
spam |WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only. |
spam |Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030 |
ham |I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today. |
spam |SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info |
spam |URGENT! You have won a 1 week FREE membership in our Â£100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18 |
ham |I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.|
ham |I HAVE A DATE ON SUNDAY WITH WILL!! |
spam |XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL |
ham |Oh k...i'm watching here:) |
ham |Eh u remember how 2 spell his name... Yes i did. He v naughty make until i v wet. |
ham |Fine if thatÂ’s the way u feel. ThatÂ’s the way its gota b |
spam |England v Macedonia - dont miss the goals/team news. Txt ur national team to 87077 eg ENGLAND to 87077 Try:WALES, SCOTLAND 4txt/Ãº1.20 POBOXox36504W45WQ 16+ |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 20 rows

In [6]:
from pyspark.sql.functions import length

In [7]:
data = data.withColumn('length',length(data['text']))

In [8]:
data.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 148|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 158|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 156|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thatÂ’s t...| 58|
 spam|England v Macedon...| 156|
+-----+--------------------+------+
only showing top 20 rows

In [9]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+--------------------+------------------+
 class| avg(length)|
+--------------------+------------------+
 ham| 71.31626943005182|
 spam|138.88487282463186|
ham	HI BABE UAWAK...| 19.0|
 ham	Yeah| 38.0|
+--------------------+------------------+

In [10]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [11]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [12]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [13]:
from pyspark.ml.classification import NaiveBayes

In [14]:
nb = NaiveBayes()

In [15]:
from pyspark.ml import Pipeline

In [16]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [17]:
cleaner = data_prep_pipe.fit(data)
clean_data = cleaner.transform(data)

In [18]:
clean_data = clean_data.select(['label','features'])
clean_data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13428,[7,10,31,6...|
 0.0|(13428,[0,24,296,...|
 1.0|(13428,[2,13,19,2...|
 0.0|(13428,[0,72,78,1...|
 0.0|(13428,[36,131,30...|
 1.0|(13428,[11,60,141...|
 0.0|(13428,[11,52,108...|
 0.0|(13428,[124,185,4...|
 1.0|(13428,[1,46,118,...|
 1.0|(13428,[0,1,13,26...|
 0.0|(13428,[18,42,119...|
 1.0|(13428,[8,17,35,8...|
 1.0|(13428,[13,28,46,...|
 0.0|(13428,[38,96,221...|
 0.0|(13428,[546,1684,...|
 1.0|(13428,[28,107,11...|
 0.0|(13428,[80,215,47...|
 0.0|(13428,[0,2,49,13...|
 0.0|(13428,[0,74,104,...|
 1.0|(13428,[4,28,33,5...|
+-----+--------------------+
only showing top 20 rows

In [19]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [20]:
spam_predictor = nb.fit(training)

In [21]:
data.printSchema()

root
-- class: string (nullable = true)
-- text: string (nullable = true)
-- length: integer (nullable = true)

In [22]:
test_results = spam_predictor.transform(testing)

In [23]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13428,[0,1,3,9,1...|[-573.86035157715...|[0.99999999999912...| 0.0|
 0.0|(13428,[0,1,5,15,...|[-999.03253599877...|[1.0,1.2107737140...| 0.0|
 0.0|(13428,[0,1,10,30...|[-874.96800907089...|[1.0,1.0324578787...| 0.0|
 0.0|(13428,[0,1,20,26...|[-969.29669435541...|[1.0,9.0588763636...| 0.0|
 0.0|(13428,[0,1,21,26...|[-1000.1120542694...|[1.0,1.3275100895...| 0.0|
 0.0|(13428,[0,1,26,88...|[-1535.6594414693...|[9.74712623838090...| 1.0|
 0.0|(13428,[0,1,31,42...|[-343.42685858192...|[1.0,1.3859478264...| 0.0|
 0.0|(13428,[0,1,69,10...|[-665.72848381277...|[1.0,4.7172534917...| 0.0|
 0.0|(13428,[0,1,415,6...|[-304.35537593431...|[0.99999999999997...| 0.0|
 0.0|(13428,[0,1,493,5...|[-321.44032213138...|[0.99999999999848...| 0.0|
 0.0|(13428,[0,1,875,1...|[-100.23388393694...|[0.99999825606033...| 0.0|
 0.0|(13428,[0,1,875,1...|[-101.91687468580...|[0.99999866478355...| 0.0|
 0.0|(13428,[0,2,3,5,6...|[-2571.0158426170...|[1.0,2.8817763035...| 0.0|
 0.0|(13428,[0,2,3,6,9...|[-3293.0530568659...|[1.0,7.5894339981...| 0.0|
 0.0|(13428,[0,2,3,8,2...|[-1615.3328566865...|[1.0,4.1338685020...| 0.0|
 0.0|(13428,[0,2,4,5,7...|[-999.13460748185...|[1.0,2.4764995210...| 0.0|
 0.0|(13428,[0,2,4,5,1...|[-2510.5788879511...|[1.0,1.8527939948...| 0.0|
 0.0|(13428,[0,2,4,7,2...|[-517.55510379104...|[1.0,1.4671422403...| 0.0|
 0.0|(13428,[0,2,4,8,1...|[-1307.5294365747...|[1.0,2.6029348518...| 0.0|
 0.0|(13428,[0,2,4,8,2...|[-569.40428827188...|[1.0,7.5973004790...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [25]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.915908408246612